<a href="https://colab.research.google.com/github/saeedahmadian/MYNLP/blob/main/TorchtextPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=de359c89588545849b931b618e627bd9792037a3f0750c15683d36ab0926ffdb
  Stored in directory: /tmp/pip-ephem-wheel-cache-1k0l_v2y/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
import spacy
import torch
import torchtext
import en_core_web_md
nlp= en_core_web_md.load()

In [3]:
import pandas as pd
df_train=pd.read_csv('train.csv',header='infer')
df_train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
"Use any of the spacy or nltk methods in any level"
"here I use nltk"
import re
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('english')
stop_words.remove('not')
def CustomTokenizerSpacy(sentence):
    sentence= re.sub('n\'t',r' not',sentence)
    sentence=re.sub(r'[^a-zA-Z0-9!?,\.]',' ',sentence)
    sentence= re.sub(r'([\?!,\.]+)',lambda m: m.group(1)[0],sentence)
    sentence=re.sub(r'[ ]+',' ',sentence)
    return [token.lemma_ for token in nlp.tokenizer(sentence) if token.lemma_ not in nlp.Defaults.stop_words]# 

def CustomTokenizerNltk(sentence):
    sentence=  re.sub(r'[^a-zA-Z0-9!?,\.\']',' ',sentence)
    sentence= re.sub(r'([\?!,\.]+)',lambda m: m.group(1)[0],sentence)
    sentence=re.sub(r'[ ]+',' ',sentence)
    sentence= re.sub('n\'t',r' not',sentence)
#     stemmer=nltk.stem.WordNetLemmatizer()
    stemmer= nltk.stem.PorterStemmer()
    return [stemmer.stem(token) for token in nltk.tokenize.word_tokenize(sentence) if token not in stop_words]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# To use the ReversibleField we need to install reverse tokenizer
!git clone https://github.com/jekbradbury/revtok.git

Cloning into 'revtok'...
remote: Enumerating objects: 145, done.
remote: Total 145 (delta 0), reused 0 (delta 0), pack-reused 145
Receiving objects: 100% (145/145), 60.97 KiB | 4.06 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [14]:
!python revtok/setup.py install 

running install
running bdist_egg
running egg_info
creating revtok.egg-info
writing revtok.egg-info/PKG-INFO
writing dependency_links to revtok.egg-info/dependency_links.txt
writing requirements to revtok.egg-info/requires.txt
writing top-level names to revtok.egg-info/top_level.txt
writing manifest file 'revtok.egg-info/SOURCES.txt'
reading manifest file 'revtok.egg-info/SOURCES.txt'
writing manifest file 'revtok.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying revtok.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying revtok.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying revtok.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying revtok.egg-info/requires.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying revtok.egg-info/top_level.

In [15]:
from torchtext.data import Field,ReversibleField
# comment_text= Field(sequential=True,use_vocab=True,
#                     lower=True,batch_first=True
#                    ,tokenize=CustomTokenizerSpacy
#                    ) # we don use use_vocab=True because we are using our own toeknizer

comment_text= ReversibleField(sequential=True,use_vocab=True,
                    lower=True,batch_first=True
                   ,tokenize=CustomTokenizerSpacy
                   ) # we don use use_vocab=True because we are using our own toeknizer
toxic= Field(sequential=False,use_vocab=False)
severe_toxic= Field(sequential=False,use_vocab=False)
obscene= Field(sequential=False,use_vocab=False)
threat= Field(sequential=False,use_vocab=False)
insult= Field(sequential=False,use_vocab=False)
identity_hate= Field(sequential=False,use_vocab=False)

In [16]:
from torchtext.data import TabularDataset
train_datafields=[('id',None),('comment_text',comment_text),('toxic',toxic),('severe_toxic',severe_toxic),
                 ('obscene',obscene),('threat',threat),('insult',insult),('identity_hate',identity_hate)]
train_set= TabularDataset(path='train.csv',format='csv',fields=train_datafields,skip_header=False)

In [17]:
# comment_text.build_vocab(train_set,vectors="glove.6B.100d")
comment_text.build_vocab(train_set)
len(comment_text.vocab)

192624

In [31]:
print("convert string \"go\" to integer : {} ".format(comment_text.vocab.stoi['<pad>']))
print("convert integer to strings : {} ".format(comment_text.vocab.itos[1]))

convert string "go" to integer : 1 
convert integer to strings : <pad> 


In [48]:
from torchtext.data import Iterator,BucketIterator
train_iter= Iterator(dataset=train_set,batch_size=1,
                           sort_key= lambda x: x.comment_text,shuffle=True)

In [49]:
batch= next(iter(train_iter))

In [53]:
[comment_text.vocab.itos[int(token)] for token in batch.comment_text[0,:]]

[' ',
 'hillary',
 'rodham',
 'clinton',
 'request',
 'greetings',
 '!',
 'a',
 'proposal',
 'talk',
 'hillary',
 'rodham',
 'clinton',
 'requested',
 '8',
 'change',
 'title',
 'article',
 ',',
 'hillary',
 'rodham',
 'clinton',
 'hillary',
 'clinton',
 '.',
 'this',
 'notification',
 'provide',
 'wikipedia',
 'canvassing',
 'appropriate',
 'notification',
 ',',
 'previously',
 'participate',
 'discussion',
 'subject',
 '.',
 'cheers',
 '!',
 't']

In [ ]:
# code from http://nlp.seas.harvard.edu/2018/04/03/attention.html 
# read text after for description of what it does
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.English))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.French) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))